In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from array_utils import load_npz, write_npz
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy

softmax = scipy.special.softmax

In [ ]:
# path = 'mnist/results/stats_0.npz'
# load_npz(path)

data = {"lol": np.array([10])}

write_npz('mnist/results', 'test.npz', data)
load_npz('mnist/results/test.npz')


In [ ]:
from metrics_lib import compute_accuracies_at_confidences

In [ ]:
thresholds = np.linspace(0, 1, 50, endpoint=False)

In [ ]:
stats = load_npz('mnist/results/svi/stats_small_28.npz')
accuracies, count = compute_accuracies_at_confidences(stats['labels'], stats['probs'], thresholds)

plt.figure(figsize=(12,6))
plt.subplot(121)
plt.plot(thresholds, count)
plt.subplot(122)
plt.plot(thresholds, accuracies)
plt.show()

In [ ]:
def generate_ensemble_stats(ensemble_dir, stat_name, ensemble_size):
    probs = None
    for i in range(ensemble_size):
        stats = load_npz(os.path.join(ensemble_dir, str(i), stat_name))
        if probs is None:
            probs = stats['probs']
        else:
            probs += stats['probs']
        print(sum(probs[0]))
    probs /= ensemble_size
    return {'probs': probs, 'labels': stats['labels']}
    
stats = generate_ensemble_stats('mnist/results/ensemble', 'stats_small_0.npz', 15)
write_npz('mnist/results/ensemble/', 'stats_small_28.npz', stats)

In [ ]:
# implement BALD approach for dropout confidence
def bald_confidence(stats):   
    probs = stats['probs']
    probs_entropy = - np.sum(probs * np.log(probs + 1e-10), axis=-1)
    
    samples = softmax(stats['logits_samples'], axis=-1)
    samples_entropy = - np.mean(np.sum(samples * np.log(samples + 1e-10), axis=-1), axis=1)
    
    confidence = -probs_entropy + samples_entropy + 1
    confidence[confidence < 0] = 0
    
    print(confidence[:10])
    print(np.max(confidence))
    print(np.min(confidence))
    return confidence
    
    
#     print(generated[0], stats['probs'][0])
    
                                   
    
cf = bald_confidence(load_npz('mnist/results/dropout/stats_28.npz'))

In [ ]:
def compute_confidences(confidences, thresholds):
    shape = (len(thresholds),)
    counts = np.zeros(shape)

#   eq = np.equal(predict_class, labels)
    for i, thresh in enumerate(thresholds):
        mask = confidences >= thresh
        counts[i] = mask.sum(-1)
    return counts

compute_confidences(cf, thresholds)

In [ ]:
ood_name = 'stats_small_28.npz'
methods = ['vanilla', 'svi', 'dropout', 'll_svi', 'll_dropout', 'ensemble']
bald_methods = ['dropout', 'll_dropout']

counts = {}

for method in methods:
    path = os.path.join('mnist/results/',method, ood_name)
    stats = load_npz(path)
    _, counts[method] = compute_accuracies_at_confidences(stats['labels'], stats['probs'], thresholds)
    
for method in bald_methods:
    path = os.path.join('mnist/results/',method, 'stats_28.npz')
    stats = load_npz(path)
    confidence = compute_confidences(bald_confidence(stats), thresholds)
    print('shape conf', confidence.shape)
    counts[method+'_bald'] = confidence

plt.figure(figsize=(10, 10))
for method, count in counts.items():
    print(method, count)
    plt.plot(thresholds, count, label=method)
plt.title('Confidence for OOD')
plt.xlabel(r'$\tau$')
plt.ylabel(r'Number of example p(y|x) > $\tau$')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
method = 'dropout'
path = os.path.join('mnist/results/',method, 'stats_28.npz')
stats = load_npz(path)
# _, counts[method] = compute_accuracies_at_confidences(stats['labels'], stats['probs'], thresholds)
stats['logits_samples'][0]